# Minimal example of reduced FaIR

The version of FaIR used in FRIDA will model the following forcing categories explicitly:
- CO2
- CH4
- N2O
- aerosol-radiation interactions
- aerosol-cloud interactions
- stratospheric water vapour from methane oxidation
- land use

These forcing categories are based on correlations to emitted species that are modelled:
- halogenated gases
- O3
- black carbon on snow

These forcing categories are supplied as external time series:
- solar
- volcanic

Excluded:
- contrails (not in AR6 WG3 scenarios)

We'll use SSP2-4.5 emissions.

## Methane lifetime

Use Leach method as we don't have all the chemical precursors. We could make a probablisitic ensemble of this.

In [ ]:
import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import xarray as xr

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

In [ ]:
scenarios = ['ssp245']

In [ ]:
df_solar = pd.read_csv('../data/solar_erf_timebounds.csv', index_col='year')

In [ ]:
df_volcanic = pd.read_csv('../data/volcanic_ERF_monthly_-950001-201912.csv')

In [ ]:
solar_forcing = np.zeros(352)
volcanic_forcing = np.zeros(352)
for i, year in enumerate(np.arange(1750, 2021)):
    volcanic_forcing[i] = np.mean(
        df_volcanic.loc[
            ((year - 1) <= df_volcanic["year"]) & (df_volcanic["year"] < year)
        ].erf
    )
volcanic_forcing[271:281] = np.linspace(1, 0, 10) * volcanic_forcing[270]
solar_forcing = df_solar["erf"].loc[1750:2101].values

In [ ]:
df_configs = pd.read_csv(
    "../data/calibrated_constrained_parameters_v1.1.0.csv",
    index_col=0,
)

In [ ]:
valid_all = df_configs.index

In [ ]:
f=FAIR()  # use Leach method for methane lifetime as we don't have all the SCLFs

In [ ]:
f.define_time(1750, 2101, 1)
f.define_scenarios(scenarios)
f.define_configs(valid_all)

In [ ]:
species, properties = read_properties('../data/species_configs_properties_reduced.csv')
f.define_species(species, properties)
f.allocate()

In [ ]:
f.species

## Fill in emissions

As we're using a custom set of input species, `fill_from_rcmip()` won't work here. So we'll use a pre-supplied dataset of netCDF emissions that are the same as RCMIP, but with the NOx emissions corrected (not that we use them here, at all).

In [ ]:
da_emissions = xr.load_dataarray("../data/ssp_emissions_1750-2500.nc")

In [ ]:
da = da_emissions.loc[
    dict(
        config="unspecified", 
        scenario="ssp245", 
        timepoints=np.arange(1750.5, 2101),
    )
]

In [ ]:
da

Fill in the emissions array matching species common to both

In [ ]:
common_species = list(set(f.species) & set(list(da.specie.to_numpy())))
common_species

In [ ]:
for specie in common_species:
    f.emissions.loc[
        dict(specie=specie)
    ] = da.loc[
        dict(specie=specie)
    ].expand_dims(dim=["scenario", "config"], axis=(1, 2)).drop("config")

In [ ]:
f.emissions

## Fill in forcing

Fill solar and volcanic forcing from external time series, and then fill in O3, BC snow and F-gases from regressions.


Forcing is on timebounds, and the regression relationships are on time points, so we'll linearly interpolate in this quick example, though Chris do whatever you did previously to derive the coefficients

In [ ]:
trend_shape = np.ones(352)
trend_shape[:271] = np.linspace(0, 1, 271)
fill(
    f.forcing,
    volcanic_forcing[:, None, None] * df_configs["scale Volcanic"].values.squeeze(),
    specie="Volcanic",
)
fill(
    f.forcing,
    solar_forcing[:, None, None] * df_configs["solar_amplitude"].values.squeeze()
    + trend_shape[:, None, None] * df_configs["solar_trend"].values.squeeze(),
    specie="Solar",
)

In [ ]:
o3_sulfur = 0.00194893
o3_ch4 = 0.00072183
o3_int = 0.048424

bcsnow_co2afolu = 0.00117162
bcsnow_sulfur = 0.00080857
bcsnow_int = -0.0024945

fgas_ch4 = 0.00059102
fgas_n2o = -0.00195974
fgas_int = 0.16398

In [ ]:
o3_timepoints = (
    o3_int + 
    o3_sulfur * f.emissions.loc[dict(specie='Sulfur', scenario='ssp245', config=valid_all[0])] +
    o3_ch4 * f.emissions.loc[dict(specie='CH4', scenario='ssp245', config=valid_all[0])]
)

In [ ]:
interpolator = interp1d(np.arange(1750.5, 2101), o3_timepoints)
o3_timebounds = np.ones(352) * np.nan
o3_timebounds[1:-1] = interpolator(np.arange(1751, 2101))
o3_timebounds[0] = o3_timepoints[0]
o3_timebounds[-1] = o3_timepoints[-1]

In [ ]:
# Ozone simulated forcing
pl.plot(f.timepoints, o3_timepoints);
pl.plot(f.timebounds, o3_timebounds);

In [ ]:
bcsnow_timepoints = (
    bcsnow_int + 
    bcsnow_sulfur * f.emissions.loc[dict(specie='Sulfur', scenario='ssp245', config=valid_all[0])] +
    bcsnow_co2afolu * f.emissions.loc[dict(specie='CO2 AFOLU', scenario='ssp245', config=valid_all[0])]
)

In [ ]:
interpolator = interp1d(np.arange(1750.5, 2101), bcsnow_timepoints)
bcsnow_timebounds = np.ones(352) * np.nan
bcsnow_timebounds[1:-1] = interpolator(np.arange(1751, 2101))
bcsnow_timebounds[0] = bcsnow_timepoints[0]
bcsnow_timebounds[-1] = bcsnow_timepoints[-1]

In [ ]:
# BC snow simulated forcing
pl.plot(f.timepoints, bcsnow_timepoints);
pl.plot(f.timebounds, bcsnow_timebounds);

In [ ]:
fgas_timepoints = (
    fgas_int + 
    fgas_n2o * f.emissions.loc[dict(specie='N2O', scenario='ssp245', config=valid_all[0])] +
    fgas_ch4 * f.emissions.loc[dict(specie='CH4', scenario='ssp245', config=valid_all[0])]
)

In [ ]:
interpolator = interp1d(np.arange(1750.5, 2101), fgas_timepoints)
fgas_timebounds = np.ones(352) * np.nan
fgas_timebounds[1:-1] = interpolator(np.arange(1751, 2101))
fgas_timebounds[0] = fgas_timepoints[0]
fgas_timebounds[-1] = fgas_timepoints[-1]

In [ ]:
# Ozone simulated forcing
pl.plot(f.timepoints, fgas_timepoints);
pl.plot(f.timebounds, fgas_timebounds);

Now fill in these forcing time series

In [ ]:
fill(f.forcing, fgas_timebounds[:, None], specie="F-gases", scenario="ssp245")
fill(f.forcing, o3_timebounds[:, None], specie="Ozone", scenario="ssp245")
fill(f.forcing, bcsnow_timebounds[:, None], specie="Light absorbing particles on snow and ice", scenario="ssp245")

## Climate response configs

These come straight from probabilistic calibration

In [ ]:
fill(f.climate_configs["ocean_heat_capacity"], df_configs.loc[:, "c1":"c3"].values)
fill(
    f.climate_configs["ocean_heat_transfer"],
    df_configs.loc[:, "kappa1":"kappa3"].values,
)
fill(f.climate_configs["deep_ocean_efficacy"], df_configs["epsilon"].values.squeeze())
fill(f.climate_configs["gamma_autocorrelation"], df_configs["gamma"].values.squeeze())
fill(f.climate_configs["sigma_eta"], df_configs["sigma_eta"].values.squeeze())
fill(f.climate_configs["sigma_xi"], df_configs["sigma_xi"].values.squeeze())
fill(f.climate_configs["seed"], df_configs["seed"])
fill(f.climate_configs["stochastic_run"], True)
fill(f.climate_configs["use_seed"], True)
fill(f.climate_configs["forcing_4co2"], df_configs["F_4xCO2"])

## Species configs

First we load up the defaults for all species that come from our datafile.

Then we override specific values for some species from the probabilistic calibration, but many are not relevant to the reduced version, and override the baseline values loaded into `species_configs_properties_reduced.csv`.

We will do a specific calibration with the reduced FaIR to get better values for this version.

In [ ]:
f.fill_species_configs('../data/species_configs_properties_reduced.csv')

# carbon cycle
fill(f.species_configs["iirf_0"], df_configs["r0"].values.squeeze(), specie="CO2")
fill(f.species_configs["iirf_airborne"], df_configs["rA"].values.squeeze(), specie="CO2")
fill(f.species_configs["iirf_uptake"], df_configs["rU"].values.squeeze(), specie="CO2")
fill(f.species_configs["iirf_temperature"], df_configs["rT"].values.squeeze(), specie="CO2")

# aerosol indirect sensitivity to SO2 emissions
fill(f.species_configs["aci_scale"], df_configs["beta"].values.squeeze())
fill(f.species_configs["aci_shape"], df_configs["shape Sulfur"].values.squeeze(), specie="Sulfur")

# double check methane concentrations and forcing doesn't suck in the calibration code

# emissions adjustments for N2O and CH4 (we don't want to make these defaults as people
# might wanna run pulse expts with these gases)
# these are 1750 emissions levels and are subtracted from the emissions time series in the concentration anomaly
# calculation.
fill(f.species_configs["baseline_emissions"], 19.019783117809567, specie="CH4")
fill(f.species_configs["baseline_emissions"], 0.08602230754, specie="N2O")

# aerosol direct sensitivity to SO2 emissions
fill(f.species_configs["erfari_radiative_efficiency"], df_configs[f"ari Sulfur"], specie="Sulfur")

# forcing scaling
for specie in [
    "CO2",
    "CH4",
    "N2O",
    "Stratospheric water vapour",
    "Land use",
]:
    fill(
        f.species_configs["forcing_scale"],
        df_configs[f"scale {specie}"].values.squeeze(),
        specie=specie,
    )

# For some reason the full time series scaling doesn't feed through, so need to scale it here for F-gases and BCSnow.
fill(
    f.forcing,
    fgas_timebounds[:, None, None] * df_configs["scale minorGHG"].values.squeeze(),
    specie="F-gases",
)
    
fill(
    f.forcing,
    bcsnow_timebounds[:, None, None] * df_configs["scale Light absorbing particles on snow and ice"].values.squeeze(),
    specie="Light absorbing particles on snow and ice",
)
# We have to come up with a way to get uncertainty in the regression coefficients for ozone that gives us 
# a roughly +/- 50% uncertainty in the present-day forcing - this will come in the calibration and sampling

# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(
    f.species_configs["baseline_concentration"],
    df_configs["co2_concentration_1750"].values.squeeze(),
    specie="CO2",
)

In [ ]:
# initial conditions
initialise(f.concentration, f.species_configs["baseline_concentration"])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)

In [ ]:
f.run()

In [ ]:
pl.plot(
    f.timebounds, 
    (
        f.temperature.loc[dict(layer=0, scenario='ssp245')] - 
        f.temperature.loc[dict(layer=0, scenario='ssp245', timebounds=np.arange(1850, 1901))].mean(axis=0)
    )
);

In [ ]:
np.percentile(
    f.temperature.loc[dict(layer=0, scenario='ssp245', timebounds=np.arange(1995, 2014))].mean(axis=0) - 
    f.temperature.loc[dict(layer=0, scenario='ssp245', timebounds=np.arange(1850, 1901))].mean(axis=0),
    (5, 50, 95)
)

In [ ]:
# present day forcing looks nice
np.percentile(
    f.forcing_sum.loc[dict(scenario='ssp245', timebounds=2019)],
    (5, 50, 95)
)

In [ ]:
pl.plot(f.timebounds, f.forcing_sum.loc[dict(scenario='ssp245')]);
pl.ylim(-2, 6.5)

In [ ]:
pl.plot(f.timebounds, f.forcing.loc[dict(specie='CO2', scenario='ssp245')]);

In [ ]:
pl.plot(f.timebounds, f.forcing.loc[dict(specie='CH4', scenario='ssp245')]);

In [ ]:
pl.plot(f.timebounds, f.forcing.loc[dict(specie='N2O', scenario='ssp245')]);

In [ ]:
pl.plot(f.timebounds, f.forcing.loc[dict(specie='Aerosol-radiation interactions', scenario='ssp245')]);

In [ ]:
pl.plot(f.timebounds, f.forcing.loc[dict(specie='Aerosol-cloud interactions', scenario='ssp245')]);

In [ ]:
pl.plot(f.timebounds, f.forcing.loc[dict(specie='Land use', scenario='ssp245')]);

In [ ]:
pl.plot(f.timebounds, f.forcing.loc[dict(specie='Ozone', scenario='ssp245')]);
# temperature feedback is switched on here - do we want it?

In [ ]:
pl.plot(f.timebounds, f.forcing.loc[dict(specie='F-gases', scenario='ssp245')]);

In [ ]:
pl.plot(f.timebounds, f.forcing.loc[dict(specie='Light absorbing particles on snow and ice', scenario='ssp245')]);

In [ ]:
pl.plot(f.timebounds, f.forcing.loc[dict(specie='Stratospheric water vapour', scenario='ssp245')]);

In [ ]:
pl.plot(f.timebounds, f.forcing.loc[dict(specie='Solar', scenario='ssp245')]);

In [ ]:
pl.plot(f.timebounds, f.forcing.loc[dict(specie='Volcanic', scenario='ssp245')]);